In [ ]:
# [tavily](https://www.tavily.com/)
# put TAVILY_API_KEY to .env
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [ ]:
# Set up Tavily tool
from llama_index.tools.tavily_research.base import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=TAVILY_API_KEY,
)

tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

In [ ]:
query = '徵象(Signs)及症狀(Symptoms)之區別?'

In [ ]:
tavily_response = tavily_tool.search(query, max_results=3)
tavily_response

In [ ]:
doc = tavily_response[0]
text = doc.text_resource.text
metadata = doc.metadata
metadata

In [ ]:
text

# search tool in an agent

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

agent = FunctionAgent(
    tools=tavily_tool_list,
    llm=OpenAI(model="gpt-5-mini"),
#    system_prompt=system_prompt,
    streaming=False,
    verbose=True,
)

In [ ]:
result = await agent.run(query)
print("-----\nResult: ")
print(result)

In [ ]:
print(tavily_tool_list[0].metadata.name)

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

agent = FunctionAgent(
    tools=tavily_tool_list,
    llm=OpenAI(model="gpt-5-mini"),
#    system_prompt=system_prompt,
    streaming=False,
    verbose=True,
)

In [ ]:
result = await agent.run(query)
print(result)

In [ ]:
result

In [ ]:
system_prompt = """
你是一個考題助理，不能直接用自己的知識回答。

任務流程：
1. 根據使用者問題，先分析並決定最重要的 **關鍵字或子問題**（必須明確列出）。
2. 對每個關鍵字或子問題，呼叫 tavily 搜尋工具來獲取相關資訊。
3. 根據搜尋回傳的內容，整理出最終答案。
   - 不允許只靠你內建的知識作答。

規則：
- 如果沒有搜尋結果，請誠實說明。
- 回答時，先列出你選定的關鍵字，再呈現答案。
"""

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

agent = FunctionAgent(
    tools=tavily_tool_list,
    llm=OpenAI(model="gpt-5-mini"),
    system_prompt=system_prompt,
    streaming=False,
    verbose=True,
)


In [ ]:
result = await agent.run(query)
print("-----\nResult: ")
print(result)

In [ ]:
result

In [ ]:
result.response

In [ ]:
result.raw

In [ ]:
len(result.tool_calls)

In [ ]:
result.tool_calls[0]

In [ ]:
for tool_call_result in result.tool_calls:
    print(f'tool_name: {tool_call_result.tool_name}')
    print(f'query: {tool_call_result.tool_kwargs}')

In [ ]:
len(tool_call_result.tool_output.blocks)

In [ ]:
tool_call_result.tool_kwargs

In [ ]:
tool_call_result

In [ ]:
from langfuse import get_client
langfuse = get_client()

# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

# Initialize LlamaIndex instrumentation
LlamaIndexInstrumentor().instrument()

In [ ]:
result = await agent.run(query)
print(result)

In [ ]:
result

In [ ]:
agent

In [ ]:
llm=OpenAI(model="gpt-5-mini")

In [ ]:
result = """我將先列出我用來檢索的關鍵字／子問題，然後給出整理後的答案（答案依據檢索到的資料，而非僅憑我內建知識）。

檢索關鍵字／子問題
1) 「徵象 症狀 區別 定義」  
2) 「signs vs symptoms difference objective subjective」  
3) 「徵象 範例 症狀 範例」  
4) 「醫學 教科書 徵象 症狀 定義」

整理後答案（依據上述搜尋結果）
- 基本定義（要點）
  - 症狀（Symptom）：為患者主觀感受或描述，只有患者本人才會知道或感受到，不能由他人直接量測或觀察到。例如：疼痛、噁心、頭暈、疲倦、心悸的感覺。（來源：多個醫療衛教網站與教科書說明）
  - 徵象（Sign）：為醫療人員或他人可客觀觀察或測量到的發現，可以被第三方確認。例如：發燒（可測到體溫）、皮疹、腫脹、血壓升高、實體檢查發現的觸痛、實驗室或影像檢查結果。簡言之：徵象是「可觀察/可量測的客觀證據」。（來源：Mayo Clinic 等醫療網站說明）

- 判別重點（簡單記憶法）
  - 症狀 = 主觀（patient-reported）  
  - 徵象 = 客觀（observer- or test-detected）

- 常見範例（配對示例）
  - 患者說「我覺得發燒、很冷」→ 症狀（feeling feverish）；測到體溫 38.5°C → 徵象（fever，客觀測量）。  
  - 患者說「肚子痛」→ 症狀；觸診時腹部壓痛 → 徵象（腹部壓痛）。  
  - 患者感到頭暈→ 症狀；眼震（nystagmus）或血壓過低測得→ 徵象。  
  - 咳嗽可同時為患者主訴（症狀），但醫護在診間觀察到持續劇烈咳嗽或吐痰帶血則為徵象；同理，尿液肉眼見血或檢驗出血尿為徵象。（來源：臨床教科書與臨床衛教資料）

- 記錄與臨床流程上的應用
  - 在病歷書寫與臨床流程（例如 SOAP 記錄法）中，通常把患者描述的主觀資訊放在 S（Subjective，症狀）欄，把檢查、生命徵象、檢驗或影像結果等客觀發現放在 O（Objective，徵象）欄。臨床診斷與處置會綜合兩者。 （來源：臨床教科書與醫療網站）

- 進一步提醒 / 邊界情況
  - 有時某些「外顯的主觀經驗」（例如患者自行看到皮疹或血尿）會被患者敘述為症狀，但當醫師或檢驗證實並測量後即成為徵象。  
  - 臨床上「症狀」與「徵象」往往共同使用以協助鑑別診斷：症狀提示要進一步檢查，徵象（尤其是可測量的檢驗或影像）則有助於確認診斷與評估病情嚴重度。 （來源：多方醫療衛教與教科書說明）

如果你需要，我可以把我檢索到的具體來源（例如醫院網站、衛福部或教科書章節連結）列出來，或給更多臨床配對範例供練習。要我列出來源嗎？"""

In [ ]:
from llama_index.core.llms import ChatMessage

In [ ]:
prompt = """你是一個知識卡生成器。  
請將輸入的資訊整理成三段，分別是：  
1. # question （原始問題，保持簡潔明確）  
2. # context （可協助用來回答原始問題的關鍵資訊，如原始問題中特殊名詞的定義）  
3. # answer （最終整合的答案，盡可能一言以蔽之，否則條列清楚，避免多餘贅述）  

請務必輸出成以下格式，不要額外加其他文字：

# question
{填入問題}

# context
{填入相關的關鍵資訊}

# answer
{填入條理清晰的答案}
"""


In [ ]:
query = '徵象(Signs)及症狀(Symptoms)之區別?'
messages_dict = [
    {'role': 'user', 'content': query},
    {'role': 'assistant', 'content': result},
    {'role': 'user', 'content': prompt}
]
messages = [ChatMessage(**msg) for msg in messages_dict]

# Print the list of ChatMessage objects to verify the conversion.
print(messages)

In [ ]:
response = llm.chat(messages)
print(response)